In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome.service import Service
# import time

# # Set up the Chrome driver
# service = Service('path/to/chromedriver')
# driver = webdriver.Chrome(service=service)

# # Load the initial page
# driver.get('https://page.kakao.com/menu/10/screen/5?tab_uid=1')
# time.sleep(2)

# # Find the element for the next week button and click it
# next_week_button = driver.find_element_by_xpath('//button[@aria-label="Next week"]')
# next_week_button.click()
# time.sleep(2)

# # Loop through the pages for each week and extract the information
# for i in range(1, 6):
#     print("Week", i)

#     # Find all the weekly boxes on the current page and extract the information
#     weekly_boxes = driver.find_elements_by_class_name('weekly-box')
#     for weekly_box in weekly_boxes:
#         title = weekly_box.find_element_by_class_name('title').text.strip()
#         author = weekly_box.find_element_by_class_name('author').text.strip()
#         synopsis = weekly_box.find_element_by_class_name('synopsis').text.strip()

#         print("Title:", title)
#         print("Author:", author)
#         print("Synopsis:", synopsis)
#         print()

#     # Click the next week button to go to the next page
#     next_week_button.click()
#     time.sleep(2)

# # Quit the browser
# driver.quit()

In [ ]:
# from bs4 import BeautifulSoup
# from selenium import webdriver
# import time
# from konlpy.tag import Okt
# from collections import Counter
# import platform
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
# import csv
# import pandas as pd

# driver=webdriver.Chrome('chrome\chromedriver.exe')

# driver.get('https://page.kakao.com/menu/10/screen/5?tab_uid=1')


# driver.find_element_by_xpath('//*[@id="S_PLAN_NO"]/option[3]').click()


# # 웹툰 요일별 Xpath
# driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div/div[1]/div/div[j]/a').click()



# data_list=[]
# stack=[]

In [1]:
import json
import openpyxl
import requests
from tqdm import tqdm
import time
from bs4 import BeautifulSoup


def epi_info(series_id, state):
    epi_url = 'https://webtoon.kakao.com/content/text/' + str(series_id)
    epi_res = requests.get(epi_url)

    soup = BeautifulSoup(epi_res.content, 'html.parser')
    result = soup.select_one('#__NEXT_DATA__').get_text()
    json_data = json.loads(result)

    series_title = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['title']
    genre = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['genre']
    authors = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['authors']
    viewCount = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['viewCount']
    likeCount = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['likeCount']
    isAdult = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['isAdult']
    synopsis = json_data['props']['initialState']['content']['contentMap'][str(series_id)]['synopsis']
    
    try:
        publisher = [value['names'][0] for value in json_data['props']['initialState']['content']['contentMap'][str(series_id)]['authorDetails'] if value['type']=='publisher'][0]
    except:
        publisher = '-'
        
    return [
        series_id,
        series_title,
        authors,
        publisher,
        isAdult,
        viewCount,
        likeCount,
        state,
        genre,
        synopsis
    ]


def get_series():
    # 연재작 수집
    state = '연재_일요일'
    url = 'https://gateway-kw.kakao.com/section/v1/timetables/days?placement=timetable_sun'

    res = requests.get(url)
    contents = res.json()['data'][0]['cardGroups'][0]['cards']

    wb = openpyxl.Workbook()
    ws = wb.active
    ws.append([
        'series_id',
        'series_title',
        'authors',
        'publisher',
        'isAdult',
        'viewCount',
        'likeCount',
        'state',
        'genre',
        'synopsis'
    ])

    error_page = []

    for series in tqdm(contents):
        series_id = series['content']['id']

        try:
            ws.append(epi_info(series_id, state))
            time.sleep(3)

        except:
            print(series_id)
            error_page.append(series_id)


    wb.save('카카오웹툰_{}.xlsx'.format(state))
    print(error_page)
    
get_series()

100%|██████████| 112/112 [06:51<00:00,  3.67s/it]


[]
